In [1]:
# This notebook needs to be run in the root folder.
import pandas as pd
from spade.candidate_gen import generate_candidate_assertions
from spade.execute_assertions import execute_candidate_assertions
from spade.label_results import prepare_for_optimizer
from spade.check_subsumes import evaluate_all_subsumes, collate_subsumption_results
from spade.optimizer import select_functions
from rich import print

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyashankar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataset_name = "medical"
from evalgen_experiments.medical.prompt_templates import TEMPLATES

prompt_template_strings = TEMPLATES
EXAMPLE = {
    "document": "Patient ID: virtassist-D2N068-VA052\nPatient Name: Brian  White\nAge: 58\nGender: male\nChief Complaint: follow-up of chronic problems\nSecondary Complaints: congestive heart failure;hypertension\n\n---------------------------------------------\nMEDICAL NOTE:\n\nCHIEF COMPLAINT\n\nFollow-up of chronic problems.\n\nHISTORY OF PRESENT ILLNESS\n\nBrian White is a 58-year-old male with a past medical history significant for congestive heart failure and hypertension, who presents today for follow-up of his chronic problems.\n\nThe patient states he has been feeling out of sorts lately. He is not sure if it is due to the change in the seasons or due to performing lots of projects and some construction on his home. He reports fatigue and lightheadedness. This has been going on for about 5 weeks. While exerting energy, he has experienced some shortness of breath and chest cramps. The patient also notes a slight cough, but he is not sure if it is just the change in seasons.\n\nHe feels bloated every once in a while. His diet has been a little bit of a struggle. They had construction on their kitchen begin over Labor Day weekend, and have been eating less healthy food as a result.\n\nRegarding his heart failure, he has been pretty good with his salt intake. He has been pretty good about his diet since the last year and is staying on top of that as much as possible. The patient has continued to utilize Lasix daily.\n\nFor his hypertension, this has been well controlled with lisinopril 20 mg a day. He has continued to monitor his blood pressure regularly.\n\nThe patient did the review of systems sheet when he checked in. He denies weight gain, swelling in the lower extremities, fevers, chills, dizziness, nausea, vomiting, and diarrhea.\n\nREVIEW OF SYSTEMS\n\n\u2022 Constitutional: Endorses fatigue. Denies fevers, chills, or weight loss.\n\u2022 Cardiovascular: Endorses chest pain or dyspnea on exertion.\n\u2022 Respiratory: Endorses cough and shortness of breath.\n\u2022 Gastrointestinal: Endorses bloating.\n\nPHYSICAL EXAMINATION\n\n\u2022 Neck: JVD 8 cm.\n\u2022 Respiratory: Rales bilateral bases.\n\u2022 Cardiovascular: 3/6 systolic ejection murmur.\n\u2022 Musculoskeletal: 1+ pitting edema bilateral lower extremities.\n\nRESULTS\n\nX-ray of the chest demonstrates a mild amount of fluid in the lungs.\n\nEchocardiogram demonstrates decreased ejection fraction of 45% and mild mitral regurgitation.\n\nASSESSMENT AND PLAN\n\nBrian White is a 58-year-old male with a past medical history significant for congestive heart failure and hypertension, who presents today for follow up of his chronic problems.\n\nCongestive heart failure.\n\u2022 Medical Reasoning: The patient reports increased fatigue, dizziness, and chest discomfort on exertion. He also exhibits some jugular venous distention, lung base crackles, and lower extremity edema on exam today. He has been compliant with his current medications but admits to dietary indiscretion lately. His recent echocardiogram demonstrated a reduced ejection fraction of 45%, as well as mitral regurgitation.\n\u2022 Additional Testing: We will order a repeat echocardiogram.\n\u2022 Medical Treatment: Increase Lasix to 80 mg daily.\n\u2022 Patient Education and Counseling: I advised the patient to monitor and record his daily weight and report those to me via the patient portal. He will contact me should he continue to experience any dyspnea.\n\nHypertension.\n\u2022 Medical Reasoning: This is well controlled based on home monitoring.\n\u2022 Medical Treatment: Continue lisinopril 20 mg daily.\n\u2022 Patient Education and Counseling: I advised him to monitor and record his blood pressures at home and report these to me via the patient portal.\n\nPatient Agreements: The patient understands and agrees with the recommended medical treatment plan.\n\n---------------------------------------------\nDIALOGUE:\n\n[doctor] hi , brian . how are you ?\n[patient] hi , good to see you .\n[doctor] it's good to see you too . so , i know the nurse told you a little bit about dax .\n[patient] mm-hmm .\n[doctor] i'd like to tell dax about you , okay ?\n[patient] sure .\n[doctor] so , brian is a 58 year old male with a past medical history significant for congestive heart failure and hypertension , who presents today for follow-up of his chronic problems . so , brian , it's been a little while i've seen you .\n[patient] mm-hmm .\n[doctor] whats , what's going on ?\n[patient] i , i just feel out of sorts lately . i do n't know if it's the change in the seasons or if we're just doing a lot of projects around the house and , and some , some construction on our own . i'm just feeling out of it . lack of , uh , energy . i'm just so tired and fatigued , and i feel kinda ... i feel lightheaded every once in a while .\n[doctor] okay . all right . um , how long has that been going on for ?\n[patient] uh , probably since labor day , so about five weeks or so .\n[doctor] okay . and , have you noticed any , like , symptoms of weight gain , like , like swollen legs , or , you know , your belly feels bloated and things like that ?\n[patient] i feel , i feel bloated every once in a while .\n[doctor] okay . all right . um , and , are you taking your , your medications ?\n[patient] uh , yes , i am .\n[doctor] okay . and , how about your diet ? are you watching your diet ?\n[patient] uh , it's been a little bit of a struggle . we began construction on our kitchen over labor day weekend , and it was ... hard to cook or prepare meals so we ate out a lot, and not always the best food out. it , it , it kind of reeked havoc , uh , so it's been maybe off a little bit .\n[doctor] okay . all right . and , how about , you know , other symptoms , like , have you had a fever or chills ?\n[patient] no .\n[doctor] okay , and any problems breathing ? do you feel short of breath ?\n[patient] uh , just when i'm doing doing the projects . again , not even lifting anything really heavy , it's just that if i'm ex- exerting any energy , i , i kinda feel it at that point .\n[doctor] okay . do you have any chest pain ?\n[patient] slight cramps . that seems to go away after about , maybe about an hour or so after i first feel it .\n[doctor] okay , and how about a cough ?\n[patient] a , a slight cough , and again , i'm not sure if it's just the change of seasons and i'm getting a cold .\n[doctor] mm-hmm . okay . all right . well , you know , for the most part , how , you know , before all of this-\n[patient] mm-hmm .\n[doctor] . how were you doing with your heart failure ? i know that we've kinda talked about you being able to watch your healthy food intake and that's been kind of a struggle in the past .\n[patient] i , i , i've actually been pretty good about that ever since . the , the , the last year , it's been a little chaotic , but i wanted to make sure i stayed on top of that .\n[doctor] okay . all right . are you excited for halloween ?\n[patient] uh , ca n't wait .\n[doctor] okay .\n[patient] our home renovations should be complete by then\n[doctor] all right , yeah , right .\n[patient] yeah .\n[doctor] and , so , lastly , for your high blood pressure , how are you doing with that ? have , are , did you buy the blood pressure cuff like i asked ?\n[patient] yeah , i , i did , and we do mon- , i , i monitor it regularly . my wife makes sure i stay on top of that , but it's been pretty good .\n[doctor] okay . all right . well , i know you did the review of systems sheet when you checked in , and you were endorsing this fatigue-\n[patient] mm-hmm .\n[doctor] . and a little dizziness and we just talked a lot about a lot of other symptoms .\n[patient] mm-hmm .\n[doctor] any other symptoms i might be missing ? nausea or vomiting , diarrhea ?\n[patient] no .\n[doctor] anything like that ?\n[patient] no .\n[doctor] okay . all right . well , i just want to go ahead and do a quick physical exam .\n[patient] mm-hmm .\n[doctor] hey , dragon ? show me the vital signs . so , looking at your vital signs here in the office , everything looks good . you know , your blood pressure and your heart rate and your oxygenation all look really good .\n[patient] mm-hmm .\n[doctor] so , i'm gon na just take a listen to a few things and check some things out , and i'll let you know what i find , okay ?\n[patient] perfect .\n[doctor] okay . so , on your physical examination , you know , i do appreciate some jugular venous distention to-\n[patient] mm-hmm .\n[doctor] to about eight centimeters . on your heart exam , i do appreciate a three out of six systolic ejection murmur , which we've heard in the past . and , on your lung exam , i do appreciate some fine crackles at the bases bilaterally , and your lower extremities have , you know , 1+ pitting edema . so , what does all that mean ? that means i think you're retaining a little bit of fluid .\n[patient] mm-hmm .\n[doctor] okay ? i wan na just go ahead and look at some of your results , okay ?\n[patient] sure .\n[doctor] hey , dragon ? show me the chest x-ray . so , looking here at the results of your chest x-ray , it does look like you have a little bit of fluid in your lungs there , and that can be just from , um , your heart failure , okay ? hey , dragon ? show me the echocardiogram . so , this is the echocardiogram that we did about four months ago , and this shows that the pumping function of your heart is a little bit reduced at 45 % , and it also shows that leaky valve , the mitral regurgitation that , that you have , okay ? um , so , let me just go over and talk about , a little bit , my assessment and my plan for you .\n[patient] mm-hmm .\n[doctor] okay ? so , for your first problem , your congestive heart failure , i think you're retaining fluid , and i wan na go ahead and increase your lasix to 80 mg once a day .\n[patient] mm-hmm .\n[doctor] i want you to weigh yourself every day . i want you to call me if you're gaining more weight .\n[patient] mm-hmm .\n[doctor] and , i certainly want you to call me if you have any other symptoms of shortness of breath , and i wan na go ahead and order another echocardiogram , okay ?\n[patient] sure .\n[doctor] hey , dragon ? order an echocardiogram .\nlastly , for your high blood pressure , it looks like you're managing it well at this time , okay ? so , i wan na go ahead and continue with the lisinopril 20 mg a day . i want you to continue to record your blood pressures at home , and report them to me in the patient portal if you see they're getting elevated , okay ?\n[patient] mm-hmm .\n[doctor] does that sound like a plan ?\n[patient] that sounds fine .\n[doctor] okay . um , i'm gon na be in touch with you after we get your test results , and we'll go from there , okay ?\n[patient] sure .\n[doctor] all right . hey , dragon , finalize the note ."
}

print(prompt_template_strings)
print(f"There are {len(prompt_template_strings)} templates.")

[
    'You are extracting insights from some medical records. The records contain a medical note and a dialogue 
between a doctor and a patient. You need to extract values for the following: Chief complaint, History of present 
illness, Physical examination, Symptoms experienced by the patient, New medications prescribed or changed, 
including dosages (if any), and Follow-up instructions (if any). Your answer should not include any personal 
identifiable information (PII) such as name, age, gender, or ID. Use "the patient" instead of their name, for 
example. Return your answer as a bullet list, where each bullet is formatted like `chief complaint: xx.` If there 
is no value for the key, the value should be `N/A`. Keep your response under 100 words.\n\n{document}',
    'You are extracting insights from some medical records. The records contain a medical note and a dialogue 
between a doctor and a patient. You need to extract values for the following: Chief complaint, History of present 
illness, Physical examination, Symptoms experienced by the patient, New medications prescribed or changed, 
including dosages (N/A if not provided), and Follow-up instructions (N/A if not provided). Your answer should not 
include any personal identifiable information (PII) such as name, age, gender, or ID. Use "the patient" instead of 
their name, for example. Return your answer as a bullet list, where each bullet is formatted like `chief complaint:
xx.` If there is no value for the key, the value should be `N/A`. Keep your response around 150 words (you may have
to summarize some extracted values to stay within the word limit).\n\n{document}'
]

There are 2 templates.

In [11]:
assertions = await generate_candidate_assertions(prompt_template_strings, EXAMPLE)

Generating assertions...
Prompt diff: --- 

+++ 

@@ -0,0 +1,8 @@

+You are extracting insights from some medical records.
+The records contain a medical note and a dialogue between a doctor and a patient.
+You need to extract values for the following: Chief complaint, History of present illness, Physical examination, Symptoms experienced by the patient, New medications prescribed or changed, including dosages (if any), and Follow-up instructions (if any).
+Your answer should not include any personal identifiable information (PII) such as name, age, gender, or ID.
+Use "the patient" instead of their name, for example.
+Return your answer as a bullet list, where each bullet is formatted like `chief complaint: xx.` If there is no value for the key, the value should be `N/A`.
+Keep your response under 100 words.
+{document}
Generating assertions...
Prompt diff: --- 

+++ 

@@ -1,8 +1,8 @@

 You are extracting insights from some medical records.
 The records contain a medical note and a di

In [15]:
# Append some stuff

beginning = """from litellm import acompletion


async def ask_llm(prompt, response, question):
    # Placeholder for asking an expert a true/false question
    # In practice, this would involve a complex implementation potentially requiring human input

    messages = [
        {
            "content": f"You are a helpful assistant. Here is the prompt:\\n\\n{prompt}\\n\\nHere is the response:\\n{response}",
            "role": "system",
        },
        {
            "content": f"{question}\\nOnly answer yes or no.",
            "role": "user",
        },
    ]

    response = await acompletion(
        model="azure/gpt-35-turbo",
        messages=messages,
    )

    # get the cost
    completion_tokens = response["usage"]["completion_tokens"]
    prompt_tokens = response["usage"]["prompt_tokens"]

    # get the response
    reply = response["choices"][0]["message"]["content"]

    if "yes" in reply.lower():
        return prompt_tokens, completion_tokens, True

    return prompt_tokens, completion_tokens, False
"""

assertion_text = beginning + assertions[1][0]

In [16]:
# Dump to functions.py

prefix = f"/Users/shreyashankar/Documents/projects/spade-experiments/evalgen_experiments/{dataset_name}"

with open(f"{prefix}/candidate_assertions.py", "w") as f:
    f.write(assertion_text)

In [23]:
# Construct reply df with cols prompt,example,response,model

# Load examples from the csv
import pandas as pd
import os
curr_dir = os.getcwd()
df = pd.read_csv(f"{curr_dir}/dataset_graded.csv")
record_list = df.to_dict(orient="records")

EXAMPLES = []
for record in record_list:
    EXAMPLES.append({
        "document": record["Var: document"],
        "id": record["Metavar: id"],
    })


reply_df = df[["Prompt", "Var: document", "Response"]].copy()
reply_df["model"] = "gpt-3"
reply_df["example"] = reply_df["Var: document"].apply(lambda x: str({"document": x}))

# Drop the Var: document col
reply_df.drop(columns=["Var: document"], inplace=True)

# Rename cols
reply_df.columns = ["prompt", "response", "model", "example"]
reply_df.head()

,prompt,response,model,example
0,You are extracting insights from some medical ...,- Chief complaint: annual exam\n- History of p...,gpt-3,"{'document': ""Patient ID: virtassist-D2N001-VA..."
1,You are extracting insights from some medical ...,- chief complaint: joint pain\n- history of pr...,gpt-3,"{'document': ""Patient ID: virtassist-D2N002-VA..."
2,You are extracting insights from some medical ...,- Chief complaint: back pain\n- History of pre...,gpt-3,"{'document': ""Patient ID: virtassist-D2N003-VA..."
3,You are extracting insights from some medical ...,- Chief complaint: back pain\n- History of pre...,gpt-3,"{'document': ""Patient ID: virtassist-D2N004-VA..."
4,You are extracting insights from some medical ...,- Chief Complaint: right middle finger pain\n-...,gpt-3,"{'document': ""Patient ID: virtassist-D2N005-VA..."


In [25]:
from evalgen_experiments.medical.candidate_assertions import ALL_FUNCTIONS

res = await execute_candidate_assertions(dataset_name, prompt_template_strings[-1], EXAMPLES, ALL_FUNCTIONS, reply_df=reply_df)


ec177d9afef45755c308b2a41da9080ac986223fbc9814648db4fb6c99053c8d
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 seconds
Sleeping for 2 

In [26]:
label_df = pd.DataFrame({"response": df["Response"].values, "label": df["grade"].values})
# Print stats
print(f"Num candidate functions: {len(ALL_FUNCTIONS)}")
print(f"Num good examples: {len(label_df[label_df['label'] == 1])}")
print(f"Num bad examples: {len(label_df[label_df['label'] == 0])}")

optimizer_dict = prepare_for_optimizer(res, label_df)

Num candidate functions: 20

Num good examples: 57

Num bad examples: 27

In [27]:
# Pickle all the results
import inspect

path_name = f"/Users/shreyashankar/Documents/projects/spade-experiments/evalgen_experiments/{dataset_name}/optimizer_input.pkl"

# optimizer_dict["K"] = K
optimizer_dict["spade_functions"] = {func.__name__: inspect.getsource(func) for func in ALL_FUNCTIONS}
import pickle

with open(path_name, "wb") as f:
    pickle.dump(optimizer_dict, f)

## Evaluating Subsumption With High Precision

In [28]:
from spade.check_subsumes import identify_subsumption_pairs
from evalgen_experiments.medical.candidate_assertions import ALL_FUNCTIONS
import pickle

dataset_name = "medical"
path_name = f"/Users/shreyashankar/Documents/projects/spade-experiments/evalgen_experiments/{dataset_name}/optimizer_input.pkl"
with open(path_name, "rb") as f:
    optimizer_dict = pickle.load(f)
K_and_pretty_df = await identify_subsumption_pairs(optimizer_dict["M"], ALL_FUNCTIONS, optimizer_dict["func_order"])

K_and_pretty_df["K"].sum() / (K_and_pretty_df["K"].shape[0] * K_and_pretty_df["K"].shape[1])

Based on the provided functions, we can identify pairs where one function implies the other or where both imply each other due to overlapping checks:

1. `assert_response_bullet_list_format` implies `assert_bullet_list_correct_format`
   - If `assert_response_bullet_list_format` returns True, it means all lines start with a "-", which is a part of what `assert_bullet_list_correct_format` checks for. However, `assert_bullet_list_correct_format` also checks for ": " in each bullet and that it ends with ".", which the former does not.

2. `assert_response_bullet_key_value_format` ⇔ `assert_bullet_list_correct_format`
   - If `assert_response_bullet_key_value_format` returns True, then `assert_bullet_list_correct_format` should too, as both check the format of bullet points with a "key: value." structure. However, note that `assert_response_bullet_key_value_format` doesn't specifically check that each bullet starts with "- ", suggesting `assert_bullet_list_correct_format` might be more str

0.035

In [47]:
# Pickle all the results
import inspect
from spade.optimizer import select_functions

path_name = f"/Users/shreyashankar/Documents/projects/spade-experiments/evalgen_experiments/{dataset_name}/optimizer_input.pkl"

optimizer_dict["K"] = K_and_pretty_df["K"]
optimizer_dict["spade_functions"] = {func.__name__: inspect.getsource(func) for func in ALL_FUNCTIONS}
import pickle

with open(path_name, "wb") as f:
    pickle.dump(optimizer_dict, f)
    
# Dump subsumption df
K_and_pretty_df["human_readable_results"].to_csv(f"/Users/shreyashankar/Documents/projects/spade-experiments/evalgen_experiments/{dataset_name}/subsumption_results.csv", index=False)

# Solve the optimization problem
optimizer_res = select_functions(path_name, tau=0.11, alpha=0.2)

# Turn into DF
import pandas as pd

optimizer_res_df = []
for method in ["spade_base", "spade_cov", "spade_sub"]:
    optimizer_res_df.append({"method": method, "ffr": optimizer_res[method]["ffr"], "example failure coverage": optimizer_res[method]["coverage"], "frac_functions_selected": optimizer_res[method]["frac_functions_selected"], "frac_non_subsumed_excluded_functions": optimizer_res[method]["frac_non_subsumed_excluded_functions"]})
    
optimizer_res_df = pd.DataFrame(optimizer_res_df)
print(optimizer_res_df)

# Pickle the results
# import pickle
# with open(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_results.pkl", "wb") as f:
#     pickle.dump(optimizer_res, f)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shreyashankar/miniforge3/envs/promptdelta/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/057c3329c3aa416fb4b7e353ef344d84-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/057c3329c3aa416fb4b7e353ef344d84-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3394 COLUMNS
At line 12934 RHS
At line 16324 BOUNDS
At line 18136 ENDATA
Problem MODEL has 3389 rows, 1811 columns and 5897 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.2 - 0.00 seconds
Cgl0002I 934 variables fixed
Cgl0003I 31 fixed, 0 tightened bounds, 36 strengthened rows, 0 substitutions
Cgl0004I processed model has 10 rows, 13 columns (13 integer (13 of which binary)) and 32 elements
Cutoff increment increa

Solution Found:

Selected Functions: [11, 19]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shreyashankar/miniforge3/envs/promptdelta/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/fe3119646868427d842b8c1bef25d377-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/fe3119646868427d842b8c1bef25d377-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4254 COLUMNS
At line 16388 RHS
At line 20638 BOUNDS
At line 22890 ENDATA
Problem MODEL has 4249 rows, 2251 columns and 7611 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 11.2282 - 0.01 seconds
Cgl0002I 1329 variables fixed
Cgl0003I 36 fixed, 0 tightened bounds, 0 strengthened rows, 5 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 4 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4 str

/Users/shreyashankar/miniforge3/envs/promptdelta/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Solution Found:

Selected Functions: [8, 11, 12, 16, 19]

method       ffr  example failure coverage  frac_functions_selected  \
0  spade_base  0.140351                  0.333333                     0.35   
1   spade_cov  0.087719                  0.259259                     0.10   
2   spade_sub  0.105263                  0.333333                     0.25   

   frac_non_subsumed_excluded_functions  
0                                   0.0  
1                                   0.2  
2                                   0.0

In [48]:
from rich import print as rprint

for k, v in optimizer_res.items():
    if "spade" in str(k):
        rprint(k)
        rprint(v["selected_function_names"])
        
        # Print the functions
        for func in v["selected_function_names"]:
            print(optimizer_dict["spade_functions"][func])
        
        rprint("\n\n\n")

spade_base

[
    'assert_adherence_to_formal_clinical_tone',
    'assert_explicit_na_if_no_follow_up',
    'assert_explicit_na_if_no_meds_changed',
    'assert_na_usage_for_missing_data',
    'assert_neutral_clinical_tone',
    'assert_response_bullet_list_format',
    'assert_use_of_the_patient_instead_of_name'
]

async def assert_adherence_to_formal_clinical_tone(response: str) -> bool:
    """
    Checks if the response maintains a formal and clinical tone appropriate for medical record analysis.
    """
    question = "Does the response maintain a formal and clinical tone appropriate for medical record analysis?"
    return await ask_llm(response, question)

async def assert_explicit_na_if_no_follow_up(response: str) -> bool:
    """
    Checks if the response explicitly states 'N/A' for follow-up instructions when no follow-up details are 
provided.
    """
    if "follow-up instructions" not in response.lower():
        return "n/a" in response.lower()
    return True  # Should return true if the segment is present, to not be consequential

async def assert_explicit_na_if_no_meds_changed(response: str) -> bool:
    """
    Checks if the response explicitly states 'N/A' for dosages when no new medications are prescribed or changed.
    """
    if "new medications prescribed or changed" not in response.lower():
        return "n/a" in response.lower()
    return True  # Should return true if the segment is present, to not be consequential

...

async def assert_explicit_na_if_no_meds_changed(response: str) -> bool:
    """
    Checks if the response explicitly states 'N/A' for dosages when no new medications are prescribed or changed.
    """
    if "new medications prescribed or changed" not in response.lower():
        return "n/a" in response.lower()
    return True  # Should return true if the segment is present, to not be consequential

async def assert_na_usage_for_missing_data(response: str) -> bool:
    """
    Checks if 'N/A' is used appropriately for data points that are not available.
    """
    response = response.lower()
    is_meds_present = "new medications prescribed or changed" in response
    is_followup_present = "follow-up instructions" in response

    meds_index = response.find("new medications prescribed or changed")
    followup_index = response.find("follow-up instructions")

    check_meds = (
        "n/a" in response
        if not is_meds_present
        else True
    )
    check_followup = (
        "n/a" in response
        if not is_followup_present
        else True
    )

    return check_meds and check_followup

async def assert_neutral_clinical_tone(response: str) -> bool:
    """
    Determines if the tone of the response is neutral, professional, and objective.
    """
    question = "Is the tone of the response neutral, professional, and objective?"
    return await ask_llm(response, question)

async def assert_response_bullet_list_format(response: str) -> bool:
    """
    Checks if the response is formatted as a bullet list.
    """
    return response.strip().startswith("-") and all(
        line.strip().startswith("-") for line in response.splitlines()
    )

async def assert_use_of_the_patient_instead_of_name(response: str) -> bool:
    """
    Checks if 'the patient' is used instead of personal names in the response.
    """

    return "the patient" in response.lower()